In [39]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [51]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay, classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [41]:
# Cargar los datos de prueba
X_test = pd.read_csv("data/X_test.csv")
y_test = pd.read_csv("data/y_test.csv").squeeze()

In [42]:
# Cargar el modelo guardado
import joblib
model_path = "best_rf_model.pkl"
modelo_cargado = joblib.load(model_path)

In [43]:
# Hacer predicciones
y_test_pred = modelo_cargado.predict(X_test)
y_test_pred_proba = modelo_cargado.predict_proba(X_test)[:, 1]

In [44]:
# Configurar el Dashboard con Streamlit
st.title("Dashboard de Resultados del Modelo")

2024-11-22 17:17:50.544 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:50.546 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [45]:
st.header("Importancia de Variables")

try:
    # Extraer los pasos del pipeline
    preprocessor = modelo_cargado.named_steps['preprocessor']  # Preprocesador del pipeline
    classifier = modelo_cargado.named_steps['classifier']  # Modelo Random Forest

    # Obtener las columnas categóricas originales
    cat_features = preprocessor.transformers_[0][2]  # Nombres de las columnas categóricas
    # Obtener las columnas codificadas tras OneHotEncoding
    cat_encoded = preprocessor.named_transformers_['cat'].get_feature_names_out(cat_features)
    # Combinar columnas codificadas y numéricas originales
    num_features = [col for col in X_test.columns if col not in cat_features]
    final_features = list(cat_encoded) + num_features

    # Obtener las importancias de las variables
    importances = classifier.feature_importances_

    # Crear un DataFrame con las importancias
    importance_df = pd.DataFrame({
        "Feature": final_features,
        "Importance": importances
    }).sort_values(by="Importance", ascending=False)

    # Mostrar el gráfico de barras
    st.bar_chart(importance_df.set_index("Feature"))

except Exception as e:
    st.write(f"Error al calcular la importancia de variables: {str(e)}")



2024-11-22 17:17:50.572 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:50.575 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:50.605 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:50.608 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [46]:
# Métricas clave
st.header("Métricas del Modelo")
report = classification_report(y_test, y_test_pred, output_dict=True)
st.write(pd.DataFrame(report).transpose())

2024-11-22 17:17:50.625 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:50.625 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:50.679 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:50.679 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [47]:
# Curva ROC
st.header("Curva ROC")
fig, ax = plt.subplots()
RocCurveDisplay.from_estimator(modelo_cargado, X_test, y_test, ax=ax)
st.pyplot(fig)


2024-11-22 17:17:50.701 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:50.703 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:50.873 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:51.036 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:51.036 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [48]:
# Importancia de Variables
st.header("Importancia de Variables")

try:
    # Verificar si hay un pipeline
    if 'preprocessor' in modelo_cargado.named_steps and 'classifier' in modelo_cargado.named_steps:
        # Extraer preprocesador y clasificador
        preprocessor = modelo_cargado.named_steps['preprocessor']
        classifier = modelo_cargado.named_steps['classifier']

        # Obtener las columnas transformadas (si se usa un preprocesador como OneHotEncoder)
        try:
            cat_features = preprocessor.transformers_[0][2]  # Nombres de las columnas categóricas
            cat_encoded = preprocessor.named_transformers_['cat'].get_feature_names_out(cat_features)  # Nombres codificados
            num_features = [col for col in X_test.columns if col not in cat_features]
            final_features = list(cat_encoded) + num_features
        except:
            # Si no hay columnas categóricas transformadas, usar las originales
            final_features = X_test.columns

    else:
        # Si no hay pipeline, usar columnas originales
        final_features = X_test.columns

    # Obtener las importancias
    importances = modelo_cargado.named_steps['classifier'].feature_importances_

    # Crear el DataFrame de importancia
    importance_df = pd.DataFrame({
        "Feature": final_features,
        "Importance": importances
    }).sort_values(by="Importance", ascending=False)

    # Mostrar gráfico
    st.bar_chart(importance_df.set_index("Feature"))

except Exception as e:
    # Si hay algún error, mostrar las importancias genéricas
    st.write(f"No se pudieron mapear las columnas, error: {str(e)}")
    importances = modelo_cargado.named_steps['classifier'].feature_importances_
    generic_features = [f"Feature {i}" for i in range(len(importances))]
    importance_df = pd.DataFrame({
        "Feature": generic_features,
        "Importance": importances
    }).sort_values(by="Importance", ascending=False)

    # Mostrar gráfico con nombres genéricos
    st.bar_char


2024-11-22 17:17:51.078 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:51.078 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:51.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:51.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [49]:
# Distribución de Errores
st.header("Distribución de Predicciones")
errors = pd.DataFrame({
    "Actual": y_test,
    "Predicho": y_test_pred,
    "Probabilidad": y_test_pred_proba
})
falsos_negativos = errors[(errors["Actual"] == 1) & (errors["Predicho"] == 0)]
falsos_positivos = errors[(errors["Actual"] == 0) & (errors["Predicho"] == 1)]
st.write(f"Falsos Negativos: {len(falsos_negativos)}")
st.write(f"Falsos Positivos: {len(falsos_positivos)}")
st.dataframe(errors.head(10))

2024-11-22 17:17:51.125 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:51.125 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:51.141 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:51.141 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:51.141 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:51.141 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:51.141 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:17:51.141 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [52]:
# Distribución de Probabilidades de Predicción
st.header("Distribución de Probabilidades de Predicción")
y_test_proba = modelo_cargado.predict_proba(X_test)[:, 1]

fig, ax = plt.subplots(figsize=(8, 6))
sns.histplot(y_test_proba, kde=True, color="blue", bins=30)
plt.title("Distribución de Probabilidades de Predicción")
plt.xlabel("Probabilidad de Avería")
plt.ylabel("Frecuencia")
st.pyplot(fig)


2024-11-22 17:19:36.629 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:19:36.631 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:19:37.065 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:19:37.326 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-22 17:19:37.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()